In [ ]:
import os

import torch
from torch import nn
from torch.optim import Adam
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

from data.dataloader import dataloader_info
from utils.utils import load_yaml_config, instantiate_from_config

from models.predictor import GRU
from models.solver import Trainer
from data.dataloader import dataloader_info
from utils.visualize import visualize_pca, visualize_tsne, visualize_kernel
from models.solver import train_prediction_model, evaluate_model

import copy

In [ ]:
# Load configurations
configs = load_yaml_config("configs/stock_adv.yaml")
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize Diffusion_TS Model
diffusion_ts = instantiate_from_config(configs['adv_model']).to(device)
batch_size = 128

In [ ]:
# dataloader
dl_info = dataloader_info(configs)
dataset = dl_info['dataset']

In [ ]:
# load original data
seq_length, feature_dim = dataset.window, dataset.feature_dim
gt_data = np.load(os.path.join(dataset.dir, f"stock_ground_truth_data_{seq_length}_train.npy"))
idx = np.random.permutation(len(gt_data))[:3000]
ori_data = gt_data[idx]

In [ ]:
test_data_norm = torch.from_numpy(np.load(os.path.join(dataset.dir, f"stock_ground_truth_data_{seq_length}_test.npy"))).to(device)
test_mean = torch.from_numpy(np.load(os.path.join(dataset.dir, f"stock_ground_truth_mean_{seq_length}_test.npy"))).to(device)
test_std = torch.from_numpy(np.load(os.path.join(dataset.dir, f"stock_ground_truth_std_{seq_length}_test.npy"))).to(device)

test_dataset = TensorDataset(test_data_norm, test_mean, test_std)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

lossfn = nn.L1Loss()
# lossfn = nn.MSELoss()


In [ ]:
gt_predictor = GRU(6, 50, 1, 2).to(device)
gt_optimizer = Adam(gt_predictor.parameters(), lr=0.001)
save_path = "output/gt_predictor_2000.pth"
# save_path = "output/predictor_2000.pth"

try :
    gt_predictor.load_state_dict(torch.load("output/gt_predictor_2000.pth"))
except:
    train_prediction_model(gt_predictor, dl_info["dataloader"], lossfn, gt_optimizer, device=device, epochs=2000, save_path=save_path)
# train_prediction_model(gt_predictor, dl_info["dataloader"],lossfn, gt_optimizer, device=device, epochs=2000, save_path=save_path)


In [ ]:
predictor = copy.deepcopy(gt_predictor)
optimizer = Adam(predictor.parameters(), lr=0.001)


In [ ]:
adv_l1, adv_l2, mape_adv, pred_y_adv, true_y = evaluate_model(predictor, dl_info["dataloader"], device)
print(f"Adv Model : \t L1 loss: {adv_l1:0.5f} \t L2 Loss : {adv_l2:0.5f} \t MAPE loss : {mape_adv:0.5f}%")

adv_l1, adv_l2, mape_adv, pred_y_adv_test, true_y_test = evaluate_model(predictor, test_loader, device)
print(f"Adv Model : \t L1 loss: {adv_l1:0.5f} \t L2 Loss : {adv_l2:0.5f} \t MAPE loss : {mape_adv:0.5f} %")


In [ ]:
diffusion_ts.set_predictor(predictor)

In [ ]:
try:
    diffusion_ts.load_state_dict(torch.load("check_points/stock/gen_24/model_5000.pth"))
except:
    trainer = Trainer(config_solver=configs["gen_solver"], model=diffusion_ts, dataloader=dl_info["dataloader"])
    trainer.train_decomp()


In [11]:
trainer = Trainer(config_solver=configs["adv_solver"], model=diffusion_ts, dataloader=dl_info["dataloader"])
trainer.train_decomp()

combiend_loss: -4996615110656.000000 l1_loss : 5492947025920.000000 fourier_loss : 2300530655232.000000 adv_loss : 12684635406336.000000:  55%|█████▍    | 1642/3000 [01:30<01:14, 18.15it/s]


KeyboardInterrupt: 

In [ ]:
adv_l1, adv_l2, mape_adv, pred_y_adv, true_y = evaluate_model(diffusion_ts.predictor, dl_info["dataloader"], device)
print(f"Adv Model : \t L1 loss: {adv_l1:0.5f} \t L2 Loss : {adv_l2:0.5f} \t MAPE loss : {mape_adv:0.5f}%")


adv_l1, adv_l2, mape_adv, pred_y_adv, true_y = evaluate_model(predictor, dl_info["dataloader"], device)
print(f"Adv Model : \t L1 loss: {adv_l1:0.5f} \t L2 Loss : {adv_l2:0.5f} \t MAPE loss : {mape_adv:0.5f}%")

In [ ]:
# generate adversarial data
adv_data = diffusion_ts.generate_mts(batch_size=1000)

In [ ]:
ori_adv_data = np.concatenate([dataset.data, adv_data])
ori_adv_data = torch.from_numpy(ori_adv_data).to(device)
ori_adv_dl = DataLoader(ori_adv_data, batch_size=batch_size)

In [ ]:
save_path = "output/predictor_2000.pth"
train_prediction_model(predictor, 
                        ori_adv_dl, 
                        lossfn, 
                        optimizer, 
                        device=device, 
                        epochs=2000, 
                        description="adversarial", 
                        save_path=save_path)
adv_l1, adv_l2, mape_adv, pred_y_adv_test, true_y_test = evaluate_model(predictor, test_loader, device)
print(f"Adv Model : \t L1 loss: {adv_l1:0.5f} \t L2 Loss : {adv_l2:0.5f} \t MAPE loss : {mape_adv:0.5f} %")


In [ ]:
for _ in range(100):
    trainer = Trainer(config_solver=configs["adv_solver"], model=diffusion_ts, dataloader=dl_info["dataloader"])
    trainer.train_decomp()
    # generate adversarial data
    adv_data = diffusion_ts.generate_mts(batch_size=1000)
    ori_adv_data = np.concatenate([dataset.data, adv_data])
    ori_adv_data = torch.from_numpy(ori_adv_data).to(device)
    ori_adv_dl = DataLoader(ori_adv_data, batch_size=batch_size)
    train_prediction_model(predictor, 
                            ori_adv_dl, 
                            lossfn, 
                            optimizer, 
                            device=device, 
                            epochs=2000, 
                            description="adversarial", 
                            save_path=save_path)
    
    adv_l1, adv_l2, mape_adv, pred_y_adv_test, true_y_test = evaluate_model(predictor, test_loader, device)
    print(f"Adv Model : \t L1 loss: {adv_l1:0.5f} \t L2 Loss : {adv_l2:0.5f} \t MAPE loss : {mape_adv:0.5f} %")